# リモート環境でのモデル学習
リモートのMachine Learning Computeでモデル学習をし、メトリックやモデルファイルをAzure Machine Learning serviceへ記録、共有します。

## Python SDKのバージョン確認

In [2]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.10


## Workspace情報の取得

In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

exp = Experiment(workspace = ws, name = "devops-notebook")

Falling back to use azure cli credentials. This fall back to use azure cli credentials will be removed in the next release. 
Make sure your code doesn't require 'az login' to have happened before using azureml-sdk, except the case when you are specifying AzureCliAuthentication in azureml-sdk.


Found the config file in: /Users/konabuta/Project/DevOpsLab/code/aml_config/config.json
Workspace name: azureml
Azure region: southeastasia
Subscription id: 9c0f91b8-eb2f-484c-979c-15848c098a6b
Resource group: azureml_sea


## Machine Learning Computeの構築

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# クラスター名称
cluster_name = "sklearn"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('同じ名前のクラスタを発見しました')
except ComputeTargetException:
    print('新しいクラスタを構築します...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',
                                                           max_nodes=2)

    # クラスタの作成
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# 詳細ログの出力 
print(compute_target.get_status().serialize())

同じ名前のクラスタを発見しました
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-02-03T07:20:21.224000+00:00', 'creationTime': '2019-02-03T07:00:06.458303+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-02-03T07:00:34.949706+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 2, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS12_V2'}


## 学習スクリプトの準備

In [5]:
from azureml.core import Experiment

experiment_name = 'remote-diabetes'
experiment = Experiment(ws, name=experiment_name)

In [6]:
project_folder = "./training"

## Estimator

In [7]:
from azureml.train.estimator import Estimator

estimator = Estimator(source_directory=project_folder,
                      compute_target=compute_target,
                      entry_script='train_diabetes.py',
                      pip_packages=['sklearn'],
                      use_gpu=False)

In [8]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: remote-diabetes,
Id: remote-diabetes_1549180419_7d890782,
Type: azureml.scriptrun,
Status: Queued)


In [9]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## モデルの登録

In [10]:
from azureml.core.model import Model

model_name = "devopsnb_remote.pkl"
model = run.register_model(model_name= model_name, 
                           model_path = 'outputs/' + model_name,
                           tags = {'area': "diabetes try modeling from remote", 'type': "regression", 'run_id' : run.id}
                          )